In [24]:
import sqlite3
import pandas as pd

In [25]:
import yfinance as yf
import time

def get_data(con, cur, stocks=[], freq='yearly'):
    
    table_names = _get_tables(cur)

    for stock in stocks:
        path = f'{stock}_{freq.upper()}'
        
        if path in table_names:
            continue
            
        try:
            time.sleep(0.3)
            
            ticker = yf.Ticker(stock)

            income_stmt = ticker.get_income_stmt(freq=freq)
            balance_sheet = ticker.get_balance_sheet(freq=freq)
            cash_flow = ticker.get_cash_flow(freq=freq)

            df = pd.concat(
                [
                    income_stmt, 
                    balance_sheet,
                    cash_flow
                ],
                axis=0, 
                join='inner'
            ).T.sort_index().drop_duplicates(keep='first')

            df.to_sql(path, con, if_exists='replace', index='Datetime')
            
            con.commit()
            
        except Exception as e:
            print(f'/nError fetching data for {stock}: {e}')

In [26]:
def get_stocks(file=None):
    
    stocks = []
    
    try:
        with open(file, 'r') as f:
            stocks = [stock.strip() for stock in f.readlines()]
            
    except FileNotFoundError:
        print(f'{file} not found/n')
    
    return stocks

In [27]:
def _get_tables(cur):
    
    cur.execute('SELECT NAME FROM sqlite_master WHERE TYPE="table"')

    return [name[0] for name in cur.fetchall()]

In [28]:
if __name__ == '__main__':
    
    db_name = 'FINANCIALS'
    file = 'data/watchlist.txt'
    freq = 'quarterly'
    
    con = sqlite3.connect(f'data/{db_name}.db')
    cur = con.cursor()
    
    stocks = get_stocks(file)
    
    get_data(con, cur, stocks, freq)